# Download IBL Data

**TODO:**
- Read in uuids of trained animals from file
- Ask Hannah how to identify trained animals

## Setup

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Connect to IBL database using DataJoint

In [2]:
import datajoint as dj

dj.config['database.host'] = 'datajoint-public.internationalbrainlab.org'
dj.config['database.user'] = 'ibldemo'
dj.config['database.password'] = 'sfn2019demo'

dj.conn()

Connecting ibldemo@datajoint-public.internationalbrainlab.org:3306


DataJoint connection (connected) ibldemo@datajoint-public.internationalbrainlab.org:3306

### Import IBL pipeline schemas and utility functions

In [3]:
from ibl_pipeline import reference, subject, action, acquisition, data, behavior, behavior_analyses, ephys, histology
from paper_behavior_functions import query_subjects

## Download data

In [4]:
# Select single session
query_session = (
    behavior.TrialSet 
    * subject.Subject 
    * subject.SubjectLab 
    * subject.SubjectProject 
    & 'subject_nickname="CSHL_015"' 
    & {'session_start_time': "2019-9-16 13:44:46"}
)
query_session

subject_uuid,session_start_time start time,subject_project,n_trials total trial numbers in this set,n_correct_trials number of the correct trials,trials_start_time start time of the trial set (seconds),trials_end_time end time of the trial set (seconds),subject_nickname nickname,sex sex,subject_birth_date birth date,ear_mark ear mark,subject_line name,subject_strain,subject_source name of source,protocol_number protocol number,subject_description,subject_ts,lab_name name of lab,subjectlab_ts,subjectproject_ts
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,ibl_neuropixel_brainwide_01,940,715,0.0,4804.95,CSHL_015,F,2019-02-19,None,C57BL/6J,None,Jax,3,None,2019-09-18 01:17:36,churchlandlab,2019-09-18 01:17:40,2019-09-18 01:17:38


In [5]:
# Select all (conclusive) trials from session
query_trials = (
    behavior.TrialSet.Trial 
    & query_session
    & 'trial_response_choice !="No Go"'
)
query_trials

subject_uuid,session_start_time start time,trial_id trial identification number,trial_start_time beginning of quiescent period time (seconds),trial_end_time end of iti (seconds),"trial_response_time Time of ""response"" in choiceworld (seconds). This is when one of the three possible choices is registered in software, will not be the same as when the mouse's movement to generate that response begins.",trial_response_choice which choice was made in choiceworld,trial_stim_on_time Time of stimulus in choiceworld (seconds),trial_stim_contrast_left contrast of the stimulus on the left,trial_stim_contrast_right contrast of the stimulus on the right,trial_feedback_time Time of feedback delivery (reward or not) in choiceworld,"trial_feedback_type whether feedback is positive or negative in choiceworld (-1 for negative, +1 for positive)","trial_rep_num the repetition number of the trial, i.e. how many trials have been repeated on this side (counting from 1)",trial_go_cue_time,trial_go_cue_trigger_time,trial_stim_prob_left probability of the stimulus being present on left,trial_reward_volume reward volume of each trial,trial_iti_duration inter-trial interval,trial_included whether the trial should be included
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,1,0.0,4.613303,2.1133000000000006,CCW,1.9543,0.0,0.0,nan,-1,None,nan,1.9544,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,2,5.0363,8.311602,5.8116,CCW,5.6365,0.0625,0.0,nan,-1,None,nan,5.6366,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,3,8.6913,10.869402000000001,9.3694,CCW,9.2041,0.0,1.0,9.3694,1,None,nan,9.2042,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,4,11.2477,14.550002000000001,13.05,CCW,12.8862,0.0,1.0,13.05,1,None,nan,12.8863,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,5,14.932,19.779201999999998,17.279200000000003,CW,15.470699999999999,0.0,0.125,nan,-1,None,nan,15.4708,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,6,20.1625,23.307302,21.807299999999998,CCW,21.6528,0.0,0.0625,21.807299999999998,1,None,nan,21.6529,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,7,23.691200000000002,27.245801999999998,25.745800000000003,CCW,25.604100000000003,0.0,1.0,25.745800000000003,1,None,nan,25.6042,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,8,27.626600000000003,32.034901999999995,30.5349,CCW,30.069200000000002,0.0,1.0,30.5349,1,None,nan,30.0693,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,9,32.4155,37.295902,34.7959,CCW,34.5372,0.0625,0.0,nan,-1,None,nan,34.5373,0.5,nan,nan,None
10fd3170-6c52-4cb6-986f-aa73704277c0,2019-09-16 13:44:46,10,37.7211,40.010002,38.51,CCW,38.2018,0.0,0.0625,38.51,1,None,nan,38.2019,0.5,nan,nan,None


In [6]:
# Fetch data
data_dict = query_trials.fetch(
    'trial_id',
    'trial_start_time',
    'trial_end_time',
    'trial_response_time',
    'trial_response_choice', 
    'trial_stim_contrast_left', 
    'trial_stim_contrast_right', 
    'trial_feedback_type',
    'trial_stim_prob_left',
    as_dict=True
)
df = pd.DataFrame(data_dict)
df

,trial_id,trial_start_time,trial_end_time,trial_response_time,trial_response_choice,trial_stim_contrast_left,trial_stim_contrast_right,trial_feedback_type,trial_stim_prob_left
0,1,0.000000,4.613303,2.113300,CCW,0.0000,0.0000,-1,0.5
1,2,5.036300,8.311602,5.811600,CCW,0.0625,0.0000,-1,0.5
2,3,8.691300,10.869402,9.369400,CCW,0.0000,1.0000,1,0.5
3,4,11.247700,14.550002,13.050000,CCW,0.0000,1.0000,1,0.5
4,5,14.932000,19.779202,17.279200,CW,0.0000,0.1250,-1,0.5
...,...,...,...,...,...,...,...,...,...
915,926,4129.107199,4131.664801,4130.164799,CCW,0.0000,0.2500,1,0.2
916,927,4132.727599,4135.308901,4133.808899,CCW,0.0000,0.0625,1,0.2
917,928,4136.366299,4138.794701,4137.294699,CCW,0.0000,0.0000,1,0.2
918,929,4139.852899,4142.384101,4140.884099,CCW,0.0000,0.0000,1,0.2


In [7]:
# Add columns for contrast (input) and report right (output)
df['signed_contrast'] = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
df['report_right'] = df['trial_response_choice'] == "CCW"

df

,trial_id,trial_start_time,trial_end_time,trial_response_time,trial_response_choice,trial_stim_contrast_left,trial_stim_contrast_right,trial_feedback_type,trial_stim_prob_left,signed_contrast,report_right
0,1,0.000000,4.613303,2.113300,CCW,0.0000,0.0000,-1,0.5,0.0000,True
1,2,5.036300,8.311602,5.811600,CCW,0.0625,0.0000,-1,0.5,-0.0625,True
2,3,8.691300,10.869402,9.369400,CCW,0.0000,1.0000,1,0.5,1.0000,True
3,4,11.247700,14.550002,13.050000,CCW,0.0000,1.0000,1,0.5,1.0000,True
4,5,14.932000,19.779202,17.279200,CW,0.0000,0.1250,-1,0.5,0.1250,False
...,...,...,...,...,...,...,...,...,...,...,...
915,926,4129.107199,4131.664801,4130.164799,CCW,0.0000,0.2500,1,0.2,0.2500,True
916,927,4132.727599,4135.308901,4133.808899,CCW,0.0000,0.0625,1,0.2,0.0625,True
917,928,4136.366299,4138.794701,4137.294699,CCW,0.0000,0.0000,1,0.2,0.0000,True
918,929,4139.852899,4142.384101,4140.884099,CCW,0.0000,0.0000,1,0.2,0.0000,True


## Save dataframe

In [8]:
data_fpath = '../data/ibl/test_1.csv'
df.to_csv(data_fpath)

## Compute psychometric curve for subject

In [11]:
#report_right = df.groupby(['signed_contrast'], as_index=False).mean()
#report_right

In [12]:
#plt.plot(report_right['signed_contrast'], report_right['report_right'], 'o')
#plt.xlabel('Signed Contrast')
#plt.ylabel('Probability reporting right')